In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import concurrent.futures
import requests
import re
import os
import config

# Use the configuration settings
TRANSFERTMARKT_TEAMS_ID = config.TRANSFERTMARKT_TEAMS_ID
FILE_NAME = config.PLAYER_FILE_NAME

In [2]:
def download_team_picture(pageSoup, team_name, folder_path="team_pictures"):

    # Find the image tag
    image_tag = pageSoup.find('img', class_='flaggenrahmen')
    if not image_tag:
        print("Image not found")
        return

    # Get the image URL
    img_url = image_tag['src']

    # Ensure the folder exists
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # Download and save the image
    response = requests.get(img_url)
    if response.status_code == 200:
        with open(os.path.join(folder_path, f"{team_name}.png"), 'wb') as file:
            file.write(response.content)
        print(f"Image saved as {team_name} in folder {folder_path}")
    else:
        print(f"Failed to download image. Status code: {response.status_code}")

def extract_players_information(pageSoup, nationality, ranking_nation, team_id):
    # Initialize an empty list to store player information
    players_info = []

    # Iterate through each player entry in the HTML
    for row in pageSoup.find_all('tr', {'class': ['even', 'odd']}):
        # Extracting each piece of information
        shirt_number = row.find('div', class_='rn_nummer').text.strip()
        player_link = row.find('a', href=re.compile(r'/profil/spieler/'))
        player_id = re.search(r'/profil/spieler/(\d+)', player_link['href']).group(1)
        player_name = player_link.text.strip()
        position = row.find_all('td')[4].text.strip()  # Adjust index based on HTML structure
        club = row.find('a', href=re.compile(r'/startseite/verein/'))['title']
        age_td = row.find_all('td', class_='zentriert')
        age = age_td[1].text.strip() if len(age_td) > 1 else None  # Adjust index based on HTML structure
        market_value_tag = row.find('td', class_='rechts hauptlink')
        market_value = market_value_tag.text.strip() if market_value_tag else None

        players_info.append({
            'PlayerID': player_id,
            'ShirtNumber': shirt_number,
            'PlayerName': player_name,
            'Age': age,
            'Nationality': nationality,
            'NationRanking': ranking_nation,
            'NationID': team_id,
            'Position': position,
            'Club': club,
            'MarketValue': market_value
        })

    return players_info

def extract_ranking_nation(pageSoup):
    # Find the <a> tag and extract the text
    a_tag = pageSoup.find('a', href="/statistik/weltrangliste")
    if a_tag:
        ranking_text = a_tag.text.strip()
        # Extract the nation classement part
        ranking_nation = ranking_text.split(' ')[-1]
        return ranking_nation
    else:
        print("Ranking information not found")
        return None
    

def scrap_national_team_transfertmarkt(team_name, team_id):
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'
    }

    try:
        page = f"https://www.transfermarkt.fr/elfenbeinkuste/kader/verein/{team_id}/saison_id/2023"
        response = requests.get(page, headers=headers)
        response.raise_for_status()  # Raise an error for bad status codes
        pageSoup = BeautifulSoup(response.content, 'html.parser')

        #download_team_picture(pageSoup, team_name)
        ranking_nation = extract_ranking_nation(pageSoup)
        players_info = extract_players_information(pageSoup, team_name, ranking_nation, team_id)
        
        return players_info

    except requests.RequestException as e:
        return f"Request error: {e}"
    except ValueError as e:
        return f"Data parsing error: {e}"
    
# Function to convert market value string to integer
def convert_market_value(value):
    if pd.isna(value):
        return np.nan  # or return 0 if you want to treat NaN as 0
    value = value.replace('€', '').strip()
    value = value.replace(',', '.')
    if 'mio.' in value:
        return float(value.replace('mio.', '')) * 1000000
    elif 'K' in value:
        return float(value.replace('K', '')) * 1000
    return float(value)

def scrap_cap_goals(player_id):
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'
    }

    try:
        page = f"https://www.transfermarkt.fr/player-name/profil/spieler/{player_id}"
        response = requests.get(page, headers=headers)
        response.raise_for_status()  # Raise an error for bad status codes

        pageSoup = BeautifulSoup(response.content, 'html.parser')

        # Find the <a> tags and extract the text
        numbers = [a.text.strip() for a in pageSoup.find_all('a', class_='data-header__content data-header__content--highlight')]

        # Assign default values if caps and goals are not found
        caps, goals = (None, None) if len(numbers) < 2 else map(int, numbers)

    except requests.RequestException as e:
        return f"Request error: {e}"
    except ValueError as e:
        return f"Data parsing error: {e}"

    return caps, goals

def fetch_caps_goals(player_id):
    caps, goals = scrap_cap_goals(player_id)
    return {'PlayerID': player_id, 'Cap': caps, 'Goal': goals}

## Extract players information for each national team

In [3]:
# Initialize an empty list to store all player data
all_players_data = []

# Loop over each team in the dictionary
for team_name, team_id in TRANSFERTMARKT_TEAMS_ID.items():
    # Scrape player data for each team
    team_players = scrap_national_team_transfertmarkt(team_name, team_id)
    
    # Check if the function returned a list (valid data)
    if isinstance(team_players, list):
        all_players_data.extend(team_players)
    else:
        print(f"Error while processing team {team_name}: {team_players}")

column_names = ['PlayerID', 'ShirtNumber', 'PlayerName', 'Age',
                'Nationality', 'NationRanking', 'NationID',
                'Position', 'Club', 'MarketValue']

In [4]:
# Create a DataFrame from the aggregated data
players_df = pd.DataFrame(all_players_data, columns=column_names)

## Extract the number of caps and goals for each player

In [5]:
# Using ThreadPoolExecutor to parallelize the process
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(fetch_caps_goals, player_id) for player_id in players_df['PlayerID']]
    players_cap_goals = [future.result() for future in concurrent.futures.as_completed(futures)]

# players_cap_goals now contains all the data

In [6]:
players_cap_goal_df = pd.DataFrame(players_cap_goals, columns=['PlayerID', 'Cap', 'Goal'])
players_df = players_df.merge(players_cap_goal_df, on='PlayerID', how='left')

In [7]:
# Transform market Value column processing
players_df['MarketValue'] = players_df['MarketValue'].apply(convert_market_value)
players_df['Age'] = pd.to_numeric(players_df['Age'], errors='coerce').fillna(0).astype(int)
players_df['MarketValue'] = pd.to_numeric(players_df['MarketValue'], errors='coerce').fillna(0).astype(float)
players_df['Cap'] = pd.to_numeric(players_df['Cap'], errors='coerce').fillna(0).astype(int)
players_df['Goal'] = pd.to_numeric(players_df['Goal'], errors='coerce').fillna(0).astype(int)

In [8]:
players_df.head()

,PlayerID,ShirtNumber,PlayerName,Age,Nationality,NationRanking,NationID,Position,Club,MarketValue,Cap,Goal
0,371816,16,Dimitry Bertaud,25,Congo,67,3854,Gardien de but,Montpellier Hérault SC,1200000.0,1,0
1,164915,1,Lionel Mpasi-Nzau,29,Congo,67,3854,Gardien de but,Rodez AF,700000.0,8,0
2,740175,21,Baggio Siadi,26,Congo,67,3854,Gardien de but,TP Mazembe,250000.0,11,0
3,203348,22,Chancel Mbemba,29,Congo,67,3854,Défenseur central,Olympique de Marseille,15000000.0,69,4
4,289432,5,Dylan Batubinsika,27,Congo,67,3854,Défenseur central,AS Saint-Étienne,1500000.0,4,0


In [9]:
rows_with_none = players_df[players_df.isna().any(axis=1)]
rows_with_none

,PlayerID,ShirtNumber,PlayerName,Age,Nationality,NationRanking,NationID,Position,Club,MarketValue,Cap,Goal


In [14]:
# Function to replace the entire value based on keywords
def replace_position(value):
    if 'Défenseur' in value or 'Arrière' in value:
        return 'Defense'
    elif 'Milieu' in value:
        return 'Midfield'
    elif 'Attaquant' in value or 'Ailier' in value or 'Avant-centre' in value or 'Deuxième' in value:
        return 'Offense'
    elif 'Gardien' in value:
        return 'Keeper'
    else:
        return value

# Apply the function to the 'Position' column
players_df['Position'] = players_df['Position'].apply(replace_position)

In [16]:
players_df.Position.unique()

array(['Keeper', 'Defense', 'Midfield', 'Offense'], dtype=object)

## Create Club table

In [17]:
club_players_df = pd.DataFrame(players_df['Club'].unique(), columns=['Club'])
club_players_df['ClubID'] = np.arange(0, len(club_players_df))

In [18]:
best_club_df = pd.read_csv("C:/Users/guygi/OneDrive/Bureau/concaf_analytics/datasets/Club.csv", encoding='ISO-8859-1')

clubs_df = club_players_df.merge(best_club_df, on='Club', how='left')
clubs_df.sort_values(by=['BestClub'], inplace=True, ascending=False)

In [19]:
clubs_df.head()

,Club,ClubID,Country,BestClub,TopLeague
194,Villarreal CF,194,ESP,True,True
80,Sevilla FC,80,ESP,True,True
90,Shakhtar Donetsk,90,UKR,True,True
221,SSC Napoli,221,IT,True,True
74,Manchester United,74,ENG,True,True


In [20]:
rows_with_none = clubs_df[clubs_df.isna().any(axis=1)]
rows_with_none.head()

,Club,ClubID,Country,BestClub,TopLeague
1,Rodez AF,1,NaN,NaN,NaN
2,TP Mazembe,2,NaN,NaN,NaN
4,AS Saint-Étienne,4,NaN,NaN,NaN
5,Hibernian FC,5,NaN,NaN,NaN
6,Simba SC,6,NaN,NaN,NaN


In [21]:
# Fill NaN values in specific columns with desired values
clubs_df['TopLeague'] = clubs_df['TopLeague'].fillna(False)
clubs_df['BestClub'] = clubs_df['BestClub'].fillna(False)
clubs_df['Country'] = clubs_df['Country'].fillna('No')

In [22]:
clubs_df.head()

,Club,ClubID,Country,BestClub,TopLeague
194,Villarreal CF,194,ESP,True,True
80,Sevilla FC,80,ESP,True,True
90,Shakhtar Donetsk,90,UKR,True,True
221,SSC Napoli,221,IT,True,True
74,Manchester United,74,ENG,True,True


## Create Nation table

In [23]:
# Grouping and calculating the required aggregates
nation_df = players_df.groupby(['NationID', 'Nationality']).agg(
    TotalPlayers=pd.NamedAgg(column='PlayerID', aggfunc='count'),
    SumMarketValue=pd.NamedAgg(column='MarketValue', aggfunc='sum'),
    MedianMarketValue=pd.NamedAgg(column='MarketValue', aggfunc='median'),
    AverageMarketValue=pd.NamedAgg(column='MarketValue', aggfunc='mean'),
    AgeAverage=pd.NamedAgg(column='Age', aggfunc='mean'),
    AverageCap=pd.NamedAgg(column='Cap', aggfunc='mean'),
    NationRanking=pd.NamedAgg(column='NationRanking', aggfunc='first')                                                                                                                                                                                                                                                                                                                                                        # Assuming ranking is the same for all players from a nation
).reset_index()

# Find the most common position for each nation
common_position = players_df.groupby(['NationID', 'Position']).size().reset_index(name='Count').sort_values(['NationID', 'Count'], ascending=[True, False])
common_position = common_position.drop_duplicates(subset=['NationID']).set_index('NationID')['Position']

# Add TopClub and CommonPosition to nation_df
nation_df = nation_df.set_index('NationID')
nation_df['CommonPosition'] = nation_df.index.map(common_position)

# Reset index
nation_df = nation_df.reset_index()

In [24]:
nation_df.head()

,NationID,Nationality,TotalPlayers,SumMarketValue,MedianMarketValue,AverageMarketValue,AgeAverage,AverageCap,NationRanking,CommonPosition
0,13485,Equatorial Guinea,27,10000000.0,200000.0,3.703704e+05,24.777778,16.333333,88,Defense
1,14238,Mauritania,27,12450000.0,150000.0,4.611111e+05,26.370370,15.703704,105,Midfield
2,14666,Tanzania,27,6775000.0,100000.0,2.509259e+05,25.333333,19.814815,121,Defense
3,3434,Cameroon,27,140450000.0,2400000.0,5.201852e+06,25.444444,19.111111,46,Defense
4,3441,Ghana,27,196100000.0,2500000.0,7.262963e+06,26.555556,18.777778,61,Offense


In [25]:
players_df = players_df.merge(clubs_df, on='Club', how='left')
players_df.drop(['Club', 'Country', 'BestClub', 'TopLeague', 'Nationality', 'NationRanking'], axis=1, inplace=True)

In [26]:
players_df.head()

,PlayerID,ShirtNumber,PlayerName,Age,NationID,Position,MarketValue,Cap,Goal,ClubID
0,371816,16,Dimitry Bertaud,25,3854,Keeper,1200000.0,1,0,0
1,164915,1,Lionel Mpasi-Nzau,29,3854,Keeper,700000.0,8,0,1
2,740175,21,Baggio Siadi,26,3854,Keeper,250000.0,11,0,2
3,203348,22,Chancel Mbemba,29,3854,Defense,15000000.0,69,4,3
4,289432,5,Dylan Batubinsika,27,3854,Defense,1500000.0,4,0,4


## Save dataframes into csv files

In [27]:
players_df.to_csv("C:/Users/guygi/OneDrive/Bureau/concaf_analytics/datasets/clean/Player.csv", encoding='utf-8-sig', index=False)
clubs_df.to_csv("C:/Users/guygi/OneDrive/Bureau/concaf_analytics/datasets/clean/Club.csv", encoding='utf-8-sig', index=False)
nation_df.to_csv("C:/Users/guygi/OneDrive/Bureau/concaf_analytics/datasets/clean/Nation.csv", encoding='utf-8-sig', index=False)